In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

In [2]:
seed = 42

In [3]:
dataset = np.load('../data/dataset.npz',)
X, y = dataset['X'], dataset['y']
print(X.shape, y.shape)

(9834, 400) (9834,)


In [4]:
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(17):
#     for i in range(5):
#         axis[label, i].imshow(X[y == label][i+5].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

In [5]:
for i in range(17):
    print(f'Label {i}: {len(X[y == i])}')

max([len(X[y == i]) for i in range(17)]) * 17


Label 0: 627
Label 1: 230
Label 2: 525
Label 3: 950
Label 4: 500
Label 5: 695
Label 6: 912
Label 7: 605
Label 8: 427
Label 9: 205
Label 10: 825
Label 11: 525
Label 12: 950
Label 13: 909
Label 14: 74
Label 15: 450
Label 16: 425


16150

#### SMOTE

In [6]:
# sm = SMOTE(random_state=seed, k_neighbors=2)
# X_res, y_res = sm.fit_resample(X.reshape(X.shape[0], -1), y)
# print(X_res.shape, y_res.shape)

# #fig, axis = plt.subplots(17, 5, figsize=(12, 48))
# for label in range(17):
#     for i in range(5):
#         axis[label, i].imshow(X_res[y_res == label][949-i].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

In [7]:
def generate_label(X, y, label, n):
    """generates n augmented images for a given label"""
    X = X[y == label]
    y = y[y == label]
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )
    X_reshaped = X.reshape(X.shape[0], 20, 20, 1)

    augmented_data = datagen.flow(X_reshaped, y, batch_size=1, seed=seed)
    X_augs, y_augs = [], []
    for i in range(n):
        X_aug, y_aug = augmented_data.__next__()
        X_aug = X_aug.flatten()
        X_augs.append(X_aug)
        y_augs.append(y_aug)
    
    X_augs = np.array(X_augs)
    y_augs = np.array(y_augs).reshape(-1)

    return np.array(X_augs), np.array(y_augs)

print(X.shape, y.shape)
X_aug, y_aug = generate_label(X, y, 1, 5)
print(X_aug.shape, y_aug.shape)

(9834, 400) (9834,)
(5, 400) (5,)


In [8]:
from preprocessing import generate_balanced_data

X_aug, y_aug = generate_balanced_data(X, y, 42)

print(X_aug.shape, y_aug.shape)
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(0,17):
#     for i in range(5):
#         axis[label, i].imshow(X_aug[y_aug == label][i].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

(16150, 400) (16150,)


In [9]:
dataset = np.load('../data/corrupt_dataset.npz',)
CX = dataset['X']
print(CX.shape)
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(0,17):
#     for i in range(5):
#         axis[label, i].imshow(X[i+label+5].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()


(935, 400)


In [11]:
class LeNet(nn.Module):
    def __init__(self, numChannels, classes):
        super(LeNet, self).__init__()

        #scales it down to 18x18 x 20
        self.conv1 = nn.Conv2d(
            in_channels=numChannels, 
            out_channels=20,
            kernel_size=(3,3), 
            )
        
        #first relu pass
        self.relu1 = nn.ReLU()
        
        #scales it down to 9x9 x 20
        self.maxpool1 = nn.MaxUnpool2d(
            kernel_size=(2,2),
            stride=(2,2)
            )

        #scales it down to 7x7 x 50 
        self.conv2 = nn.Conv2d(
            in_channels=20, 
            out_channels=50,
            kernel_size=(3,3), 
            )

        #second relu pass
        self.relu2 = nn.ReLU()

        #scales it down to 3x3 x 50
        self.maxpool2 = nn.MaxUnpool2d(
            kernel_size=(2,2),
            stride=(2,2),
            )

        #takes the 3x3x50 = 450
        self.fc1 = nn.Linear(
            in_features=450,
            out_features=500,
            )
        
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(
            in_features=500,
            out_features=classes,
        )
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def foreward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = nn.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)
        output = self.logsoftmax(x)
        return output
            

In [ ]:
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10

TRAIN_SPLIT = 0.80
TEST_SPLIT = 1 - TRAIN_SPLIT

device = torch.device("cpu")


